In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import torch
from tqdm.notebook import tqdm, trange
from viz_x import viz_pc, viz_sdf, viz_code, viz_x_img, viz_x

from ddpm import LatentDiffusionModel, CustomDataset, get_ddpm_scheduler_variables, extract

timesteps = 1000
timesteps, betas, alphas, alphas_cumprod, alphas_cumprod_prev, sqrt_recip_alphas, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, posterior_variance = get_ddpm_scheduler_variables(timesteps=timesteps)

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
torch.manual_seed(1984)

Using device: cuda:3


In [2]:
from init import get_AEmodel_cfg
from core.models import get_model
category = "mugs"
config_Only_model = get_AEmodel_cfg()
ModelClass = get_model(config_Only_model["model"]["model_name"])
model = ModelClass(config_Only_model)
ckpt_path = f"/home/ziran/se3/EFEM/weights/{category}.pt"
ckpt = torch.load(ckpt_path)
model.network.load_state_dict(ckpt['model_state_dict'])
model.network = model.network.to(device)

codebook_path = f"/home/ziran/se3/EFEM/cache/mugs.npz"
train_ds = CustomDataset(codebook_path, normalization=True)

bs = 3

z_so3 (149, 256, 3)
z_inv (149, 256)
center (149, 1, 3)
scale (149,)
z_so3_proj (149, 256, 3)
z_so3_basis (149, 3, 3)
z_so3_var (149, 3)
bbox (149, 3)
bbox_c (149, 3)
pcl (149, 5000, 3)
cls (149,)
========== Data Normalization BEGIN ==========
z_so3_mean tensor(0.0039)
z_so3_std tensor(0.0359)
z_inv_mean tensor(-0.0006)
z_inv_std tensor(0.0132)
========== Data Normalization FINISHED ==========


## sdf decoder

In [21]:
N = 11
di = 1
normal_params = train_ds.normal_params
x_gt = train_ds[2][0].unsqueeze(0).repeat(bs, 1, 1).to(device=device, dtype=torch.float32)

pred_scale = (torch.ones(bs) + 0.2).to(device)
pred_center = torch.zeros(bs, 1, 3).to(device)

space_dim = [N, N, N]
x = np.linspace(-di, di, space_dim[0])
y = np.linspace(-di, di, space_dim[1])
z = np.linspace(-di, di, space_dim[2])


In [22]:
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
viz_query = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=-1)
viz_query

array([[-1. , -1. , -1. ],
       [-1. , -1. , -0.8],
       [-1. , -1. , -0.6],
       ...,
       [ 1. ,  1. ,  0.6],
       [ 1. ,  1. ,  0.8],
       [ 1. ,  1. ,  1. ]])

In [23]:
viz_query = torch.tensor(viz_query,dtype=torch.float32).to(device)
viz_query = viz_query.repeat(bs, 1, 1)
with torch.no_grad():
    pred_so3_feat, pred_inv_feat = x_gt[:,:,:3].detach(), x_gt[:,:,3].detach()
    if normal_params is not None:
        pred_so3_feat = pred_so3_feat * normal_params["z_so3_std"] + normal_params["z_so3_mean"]
        pred_inv_feat = pred_inv_feat * normal_params["z_inv_std"] + normal_params["z_inv_mean"]
    embedding = {
        "z_so3": pred_so3_feat, # [B, 256, 3]
        "z_inv": pred_inv_feat, # [B, 256]
        "s": pred_scale, # [B]
        "t": pred_center, # [B, 1, 3]
    }
    sdf_gt = model.network.decode(viz_query, None, embedding, return_sdf=False)  

In [26]:
model.network.sdf2occ_factor

-1.0

In [30]:
type(sdf_gt)
torch.sigmoid(sdf_gt.logits).shape

torch.Size([3, 1331])

In [19]:
sdf_gt.shape
print(sdf_gt.min(), sdf_gt.max(), sdf_gt.mean())

tensor(-0.0088, device='cuda:3') tensor(0.7683, device='cuda:3') tensor(0.4781, device='cuda:3')


In [ ]:
(sdf_gt < 1e-4).sum()/3000